# seq2seq model with attention for language translation
- notes for [online tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb) and [code](https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation)